In [ ]:
# Kod snutt till labb_1 om jag hellre vill plotta som en funktion

def plot_category_vs_cardio(df, category, title, ax, rotate_xticks=False, palette=None):

    percent = df.groupby(category)['cardio'].mean() * 100
    sns.barplot(x=percent.index, y=percent.values, ax=ax, palette=palette)
    ax.set_title(title)
    ax.set_ylabel('Andel positiva (%)')
    if rotate_xticks:
        ax.set_xticks(range(len(percent.index)))  # Sätt antalet ticks
        ax.set_xticklabels(percent.index, rotation=45)

# Skapa en figur med subplots
plt.figure(figsize=(15, 12))

# Plot 1: Blodtryckskategorier
plt.subplot(2, 2, 1)
plot_category_vs_cardio(df, 'blood_pressure_category', 'Blodtryckskategorier vs Andel Positiva', 
                        plt.gca(), rotate_xticks=True, palette='viridis')

# Plot 2: BMI-kategorier
plt.subplot(2, 2, 2)
plot_category_vs_cardio(df, 'BMI_category', 'BMI-kategorier vs Andel Positiva', 
                        plt.gca(), rotate_xticks=True)

# Plot 3: Blodsocker
plt.subplot(2, 2, 3)
plot_category_vs_cardio(df_gluc, 'gluc_category', 'Blodsocker vs Andel Positiva', 
                        plt.gca())

# Plot 4: Alkohol
plt.subplot(2, 2, 4)
plot_category_vs_cardio(df, 'alco', 'Alkohol vs Andel Positiva', 
                        plt.gca())
plt.xticks(ticks=[0, 1], labels=['Inte alkoholkonsumtion', 'Alkoholkonsumtion'])


plt.tight_layout()
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
# Train-validation-test split, punkt 1 i välj modell
# Dela upp df1
X_df1 = df1.drop('cardio', axis=1) # Feauters
y_df1 = df1['cardio'] # Target

X_train_df1, X_test_df1, y_train_df1, y_test_df1 = train_test_split(X_df1, y_df1, test_size=0.2, random_state=42)
X_train_df1, X_val_df1, y_train_df1, y_val_df1 = train_test_split(X_train_df1, y_train_df1, test_size=0.25, random_state=42)

# Dela upp df2
X_df2 = df2.drop('cardio', axis=1)
y_df2 = df2['cardio']

X_train_df2, X_test_df2, y_train_df2, y_test_df2 = train_test_split(X_df2, y_df2, test_size=0.2, random_state=42)
X_train_df2, X_val_df2, y_train_df2, y_val_df2 = train_test_split(X_train_df2, y_train_df2, test_size=0.25, random_state=42)

# Visa storleken på varje del
print("Storlekar för df1:")
print(f"Träningsdata: {X_train_df1.shape}, Valideringsdata: {X_val_df1.shape}, Testdata: {X_test_df1.shape}")
print("\nStorlekar för df2:")
print(f"Träningsdata: {X_train_df2.shape}, Valideringsdata: {X_val_df2.shape}, Testdata: {X_test_df2.shape}")

In [ ]:
from sklearn.preprocessing import StandardScaler
# Skala dataset punkt 2 i välj modell
scaler_standard = StandardScaler()
X_train_df1_std = scaler_standard.fit_transform(X_train_df1)
X_val_df1_std = scaler_standard.transform(X_val_df1)
X_test_df1_std = scaler_standard.transform(X_test_df1)

from sklearn.preprocessing import MinMaxScaler

scaler_minmax = MinMaxScaler()
X_train_df1_norm = scaler_minmax.fit_transform(X_train_df1)
X_val_df1_norm = scaler_minmax.transform(X_val_df1)
X_test_df1_norm = scaler_minmax.transform(X_test_df1)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.pipeline import Pipeline

# Funktion för att träna och evaluera Random Forest
def train_evaluate_rf(df):
    # Train-validation-test split
    X = df.drop('cardio', axis=1)
    y = df['cardio']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)
    
    # Skapa pipeline med skalning och modell
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', RandomForestClassifier(random_state=42))
    ])
    
    # Definiera hyperparametrar
    param_grid = {
        'model__n_estimators': [10, 20, 30],
        'model__max_depth': [None, 10, 20],
        'model__min_samples_split': [2, 5, 10],
        'model__min_samples_leaf': [1, 2]
    }
    
    # GridSearchCV
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    # Prediction och evaluation
    y_pred = grid_search.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    
    # Returnera resultat
    return {
        'best_params': grid_search.best_params_,
        'accuracy': accuracy,
        'f1_score': f1
    }

# Kör Random Forest på båda dataseten
result_df1 = train_evaluate_rf(df1)
result_df2 = train_evaluate_rf(df2)

# Visa resultat
print("Resultat för df1:")
print(result_df1)
print("\nResultat för df2:")
print(result_df2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Definiera parametergrid, för RandomForestClassifier:
param_grid_rf = {
    'n_estimators': [10, 20, 30], # Antal träd i skogen
    'max_depth': [None, 10, 20, 30], # Maxdjup för varje träd, (None: träden växer tills alla blad är rena.)
    'min_samples_split': [2, 5, 10] # Minsta antal sampel som krävs.
}

# Skapa modell
rf_model = RandomForestClassifier(random_state=42)

# Använd funktionen
best_rf_model, best_rf_params = perform_grid_search(rf_model, param_grid_rf, X1_train_standard, y1_train)
print("Bästa parametrar för RandomForest:", best_rf_params)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Definiera parametergrid, för LogisticRegression
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

# Skapa modell
lr_model = LogisticRegression(random_state=42)

# Använd funktionen
best_lr_model, best_lr_params = perform_grid_search(lr_model, param_grid_lr, X1_train_standard, y1_train)
print("Bästa parametrar för LogisticRegression:", best_lr_params)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

# Definiera parametergrid för GradienBoostingClassifier
param_grid_gb = {
    'n_estimators': [10, 20, 30],
    'learning_rate': [0.01, 0.1, 1],
    'max_depth': [3, 5, 10]
}

# Skapa modell
gb_model = GradientBoostingClassifier(random_state=42)

# Använd GridSearchCV
best_gb_model, best_gb_params = perform_grid_search(gb_model, param_grid_gb, X1_train_standard, y1_train)
print("Bästa parametrar för GradientBoosting:", best_gb_params)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

# Definiera parametergrid för KneighborsClassifier
param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9],  # Antal grannar
    'weights': ['uniform', 'distance'],  # Viktning: 'uniform' eller 'distance'
    'p': [1,2] # Avståndsmått: 1 för Manhattan, 2 för Euklidiskt avstånd
}

knn_model = KNeighborsClassifier()

best_knn_model, best_knn_params = perform_grid_search(knn_model, param_grid_knn, X1_train_standard, y1_train)
print("Bästa parametrar för KNN:", best_knn_params)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

def train_and_evaluate_model(model, param_grid, X_train, y_train, X_val, y_val, X_test, y_test, model_name):
    """
    Tränar en modell med GridSearchCV, gör förutsägelser på validerings- och testdata,
    och returnerar den bästa modellen samt noggrannheten på validerings- och testdata.
    
    :param model: Maskininlärningsmodellen (t.ex. LogisticRegression).
    :param param_grid: Dictionary med hyperparametrar att testa.
    :param X_train: Träningsdata (features).
    :param y_train: Träningsdata (target).
    :param X_val: Valideringsdata (features).
    :param y_val: Valideringsdata (target).
    :param X_test: Testdata (features).
    :param y_test: Testdata (target).
    :param model_name: Namn på modellen (för dokumentation).
    :return: Den bästa modellen, noggrannhet på valideringsdata, noggrannhet på testdata.
    """
    # Utför GridSearchCV
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    
    # Hämta den bästa modellen
    best_model = grid_search.best_estimator_
    
    # Gör förutsägelser på validerings- och testdata
    y_val_pred = best_model.predict(X_val)
    y_test_pred = best_model.predict(X_test)
    
    # Beräkna noggrannhet på validerings- och testdata
    val_accuracy = accuracy_score(y_val, y_val_pred)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    
    return best_model, val_accuracy, test_accuracy

In [ ]:
from sklearn.linear_model import LogisticRegression


# Skapa modell
log_reg_model = LogisticRegression(random_state=42, solver='liblinear')

# Använd funktionen
best_log_reg_model, log_reg_val_accuracy, log_reg_test_accuracy = train_and_evaluate_model(
    model=log_reg_model,
    param_grid=param_grid_logreg,
    X_train=X1_train_standard,
    y_train=y1_train,
    X_val=X1_val_standard,
    y_val=y1_val,
    X_test=X1_test_standard,
    y_test=y1_test,
    model_name="Logistic Regression"
)


# Skapa modell
rf_model = RandomForestClassifier(random_state=42)

# Använd funktionen
best_rf_model, rf_val_accuracy, rf_test_accuracy = train_and_evaluate_model(
    model=rf_model,
    param_grid=param_grid_rf,
    X_train=X1_train_standard,
    y_train=y1_train,
    X_val=X1_val_standard,
    y_val=y1_val,
    X_test=X1_test_standard,
    y_test=y1_test,
    model_name="Random Forest"
)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
# Skapa modell för RandomForestClassifier:
rf_model = RandomForestClassifier(random_state=42)

# Använd perform_grid_search-funktionen
best_rf_model, best_rf_params, rf_metrics = perform_grid_search(
    model=rf_model,
    param_grid=param_grid_rf,
    X_train=X1_train_standard,
    y_train=y1_train,
    X_val=X1_val_standard,
    y_val=y1_val,
    model_name= "Random Forest"
)

# Skriv ut resultaten
print("Bästa RandomForest-modell:")
print(best_rf_model)
print("\nBästa parametrar för RandomForest:")
print(best_rf_params)
print("\nUtvärderingsmått för RandomForest:")
print(rf_metrics)

In [ ]:
# Definiera parametergrid, för DecisionTreeClassifier
param_grid_dt = {
    'max_depth': [3, 5, 10],  # Maxdjup för trädet
    'min_samples_split': [2, 5, 10],  # Minsta antal sampel för att dela en nod
    'min_samples_leaf': [1, 2, 4]  # Minsta antal sampel i ett löv
}

from sklearn.tree import DecisionTreeClassifier
# Skapa modell
dt_model = DecisionTreeClassifier(random_state=42)

# Träna och utvärdera DecisionTreeClassifier på df1 - Standardization
best_dt_model_df1, best_dt_params_df1, dt_metrics_df1 = perform_grid_search(
    model=dt_model,
    param_grid=param_grid_dt,
    X_train=X1_train_standard,
    y_train=y1_train,
    X_val=X1_val_standard,
    y_val=y1_val,
    model_name="Decision Tree Standardization (df1)"
)

# Träna och utvärdera DecisionTreeClassifier på df2 - Standardization
best_dt_model_df2, best_dt_params_df2, dt_metrics_df2 = perform_grid_search(
    model=dt_model,
    param_grid=param_grid_dt,
    X_train=X2_train_standard,
    y_train=y2_train,
    X_val=X2_val_standard,
    y_val=y2_val,
    model_name="Decision Tree Standardization (df2)"
)

# Träna och utvärdera DecisionTreeClassifier på df1 - Normalization
best_dt_model_df1_minmax, best_dt_params_df1_minmax, dt_metrics_df1_minmax = perform_grid_search(
    model=dt_model,
    param_grid=param_grid_dt,
    X_train=X1_train_minmax,
    y_train=y1_train,
    X_val=X1_val_minmax,
    y_val=y1_val,
    model_name="Decision Tree Normalization (df1)"
)

# Träna och utvärdera DecisionTreeClassifier på df2 - Normalization
best_dt_model_df2_minmax, best_dt_params_df2_minmax, dt_metrics_df2_minmax = perform_grid_search(
    model=dt_model,
    param_grid=param_grid_dt,
    X_train=X2_train_minmax,
    y_train=y2_train,
    X_val=X2_val_minmax,
    y_val=y2_val,
    model_name="Decision Tree Normalization (df2)"
)

In [ ]:
print("Resultat för Decision Tree på standardiserad data (df1):")
print(f"Bästa parametrar: {best_dt_params_df1}")
print(f"Noggrannhet: {dt_metrics_df1['accuracy']}")
print(f"Precision: {dt_metrics_df1['precision']}")
print(f"Recall: {dt_metrics_df1['recall']}")
print(f"F1-poäng: {dt_metrics_df1['f1_score']}")

print("\nResultat för Decision Tree på standardiserad data (df2):")
print(f"Bästa parametrar: {best_dt_params_df2}")
print(f"Noggrannhet: {dt_metrics_df2['accuracy']}")
print(f"Precision: {dt_metrics_df2['precision']}")
print(f"Recall: {dt_metrics_df2['recall']}")
print(f"F1-poäng: {dt_metrics_df2['f1_score']}")

print("\nResultat för Decision Tree på normaliserad data (df1):")
print(f"Bästa parametrar: {best_dt_params_df1_minmax}")
print(f"Noggrannhet: {dt_metrics_df1_minmax['accuracy']}")
print(f"Precision: {dt_metrics_df1_minmax['precision']}")
print(f"Recall: {dt_metrics_df1_minmax['recall']}")
print(f"F1-poäng: {dt_metrics_df1_minmax['f1_score']}")

print("\nResultat för Decision Tree på normaliserad data (df2):")
print(f"Bästa parametrar: {best_dt_params_df2_minmax}")
print(f"Noggrannhet: {dt_metrics_df2_minmax['accuracy']}")
print(f"Precision: {dt_metrics_df2_minmax['precision']}")
print(f"Recall: {dt_metrics_df2_minmax['recall']}")
print(f"F1-poäng: {dt_metrics_df2_minmax['f1_score']}")

In [1]:
# Hur jag hade det från början innan nya testet:

# Definiera parametergrid för RandomForest
param_grid_rf = {
    'n_estimators': [10, 20, 30], # Antal träd i skogen
    'max_depth': [5, 10, 20], # Maxdjup för varje träd.
    'min_samples_split': [2, 5, 10, 15], # Minsta antal sampel för att dela en nod
    'min_samples_leaf': [1, 2, 4] # Minsta antal sampel i ett löv
}

# Definiera parametergrid, för LogisticRegression
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10], #Regulariseringsstyrka
    'penalty': ['l1', 'l2'], # Strafftyp
    'solver': ['saga'] # Optimeringsteknik
}



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd 
import os


# Skapar en funktion för att utföra GridsearchCV
def perform_grid_search(model, param_grid, X_train, y_train, X_val, y_val, X_test, y_test, model_name, cv=5, scoring='accuracy', save_results=False, filename='model_results.csv'):
    """
    Utför GridSearchCV för en given modell och parametergrid, tränar modellen,
    gör förutsägelser på valideringsdata och beräknar utvärderingsmått.
    
    :param model: Maskininlärningsmodellen (t.ex. RandomForestClassifier).
    :param param_grid: Dictionary med hyperparametrar att testa.
    :param X_train: Träningsdata (features).
    :param y_train: Träningsdata (target).
    :param X_val: Valideringsdata (features).
    :param y_val: Valideringsdata (target).
    :param X_test: Testdata (features).
    :param y_test: Testdata (target).
    :param cv: Antal folds för korsvalidering (default: 5).
    :param scoring: Mätvärde för att välja bästa modell (default: 'accuracy').
    :param save_results: Om True, spara resultaten till en CSV-fil (default: False).
    :param filename: Namnet på CSV-filen (default: 'model_results.csv').
    :return: Den bästa modellen, dess parametrar och utvärderingsmått.
    """
    # Utför GridSearchCV
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring=scoring)
    grid_search.fit(X_train, y_train)
    
    # Hämta den bästa modellen och dess parametrar
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    # Gör förutsägelser på valideringsdata
    y_val_pred = best_model.predict(X_val)
    y_test_pred = best_model.predict(X_test)
    
    # Beräkna utvärderingsmått
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_precision = precision_score(y_val, y_val_pred)
    val_recall = recall_score(y_val, y_val_pred)
    val_f1 = f1_score(y_val, y_val_pred)
    
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred)
    test_recall = recall_score(y_test, y_test_pred)
    test_f1 = f1_score(y_test, y_test_pred)
    
    # Skapa resultaten
    evaluation_metrics = {
        'val_accuracy': val_accuracy,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1,
        'test_accuracy': test_accuracy,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'test_f1': test_f1
    }
    
    # Spara resultaten om save_results är True
    if save_results:
        results = {
            'Model': [model_name],
            'Best Parameters': [str(best_params)],
            'Validation Accuracy': [val_accuracy],
            'Test Accuracy': [test_accuracy],
            'Validation Precision': [val_precision],
            'Test Precision': [test_precision],
            'Validation Recall': [val_recall],
            'Test Recall': [test_recall],
            'Validation F1': [val_f1],
            'Test F1': [test_f1]
        }
        results_df = pd.DataFrame(results)

        # Kontrollera om filen redan finns
        if os.path.exists(filename):
            existing_df = pd.read_csv(filename)
            # Kontrollera om modellen redan finns i filen
            if model_name not in existing_df['Model'].values:
                # Lägg till nya resultat om modellen inte redan finns
                results_df.to_csv(filename, mode='a', header=False, index=False)
                print(f"Nya resultat för '{model_name}' sparades i {filename}.")
            else:
                print(f"Modellen '{model_name}' finns redan i filen. Inga nya resultat sparades.")
        else:
            # Skapa en ny fil om den inte finns
            results_df.to_csv(filename, mode='w', header=True, index=False)
            print(f"Ny fil skapad: {filename}")
    
    return best_model, best_params, evaluation_metrics

In [ ]:
    # Skapa modeller med de bästa parametrarna från tidigare resultat för df2
    rf_best = RandomForestClassifier(
        max_depth=10, 
        min_samples_leaf=4, 
        min_samples_split=10, 
        n_estimators=30,
        random_state=42
    )
    
    lr_best = LogisticRegression(
        C=100, 
        penalty='l2', 
        solver='saga',
        random_state=42
    )
    
    gb_best = GradientBoostingClassifier(
        learning_rate=0.1, 
        max_depth=5, 
        n_estimators=30,
        random_state=42
    )
    
    knn_best = KNeighborsClassifier(
        n_neighbors=9, 
        p=1, 
        weights='uniform'
    )

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd 
import os

# Ensemble modell med VotingClassifier
def create_ensemble_model(X2_train_minmax, y2_train, X2_val_minmax, y2_val, X2_test_minmax, y2_test, dataset_name="df2"):
    """
    Skapar en ensemble-modell med VotingClassifier baserat på de bästa parametrarna
    från tidigare träning. Använder 'hard' voting (majoritetsröstning).
    
    Args:
        X2_train_minmax, y2_train: Träningsdata
        X2_val_minmax, y2_val: Valideringsdata
        X2_test_minmax, y2_test: Testdata
        dataset_name: Namnet på datasetet (default: "df2")
    
    Returns:
        ensemble_model: Den tränade ensemble-modellen
        accuracy: Noggrannhet på testdata
    """
    # Ladda resultat från CSV-filen
    results_df = pd.read_csv('model_results.csv')
    
    # Filtrera resultat specifikt för df2
    df2_results = results_df[results_df['Model'].str.contains(dataset_name)]
    
    # Extrahera bästa parametrar för varje modell
    rf_params = eval(df2_results[df2_results['Model'].str.contains('Random Forest')]['Best Parameters'].values[0])
    lr_params = eval(df2_results[df2_results['Model'].str.contains('Logistic Regression')]['Best Parameters'].values[0])
    gb_params = eval(df2_results[df2_results['Model'].str.contains('Gradient Boosting')]['Best Parameters'].values[0])
    knn_params = eval(df2_results[df2_results['Model'].str.contains('KNN')]['Best Parameters'].values[0])
    
    # Lägg till random_state för modeller som stödjer det
    rf_params['random_state'] = 42
    lr_params['random_state'] = 42
    gb_params['random_state'] = 42
    
    # Skapa modeller med de extraherade parametrarna
    rf_best = RandomForestClassifier(**rf_params)
    lr_best = LogisticRegression(**lr_params)
    gb_best = GradientBoostingClassifier(**gb_params)
    knn_best = KNeighborsClassifier(**knn_params)
    
    # Kombinera tränings- och valideringsdata för att träna den slutliga modellen
    X2_train_combined = pd.concat([pd.DataFrame(X2_train_minmax), pd.DataFrame(X2_val_minmax)])
    y2_train_combined = pd.concat([pd.Series(y2_train), pd.Series(y2_val)])
    
    # Skapa VotingClassifier
    ensemble_model = VotingClassifier(
        estimators=[
            ('rf', rf_best),
            ('lr', lr_best),
            ('gb', gb_best),
            ('knn', knn_best)
        ],
        voting='hard'  # Använd majoritetsröstning
    )
    
    # Träna ensemble-modellen på kombinerad tränings- och valideringsdata
    print(f"Tränar ensemble-modell med VotingClassifier på {dataset_name}...")
    
    ensemble_model.fit(X2_train_combined, y2_train_combined)
    
    # Skapa en dictionary för att lagra individuella modeller
    individual_models = {
        'Random Forest': rf_best,
        'Logistic Regression': lr_best,
        'Gradient Boosting': gb_best,
        'KNN': knn_best
    }
    
    # Utvärdera individuella modeller och ensemble-modellen
    results = {'Modell': [], 'Noggrannhet': []}
    
    # Utvärdera individuella modeller
    print("\nUtvärderar individuella modeller på df2-datasetet...")
    for name, model in individual_models.items():
        model.fit(X2_train_combined, y2_train_combined)
        y2_pred = model.predict(X2_test_minmax)
        acc = accuracy_score(y2_test, y2_pred)
        results['Modell'].append(name)
        results['Noggrannhet'].append(acc)
        print(f"{name}: {acc:.4f}")
    
    # Utvärdera ensemble-modellen
    y2_test_pred = ensemble_model.predict(X2_test_minmax)
    test_accuracy = accuracy_score(y2_test, y2_test_pred)
    
    # Lägg till ensemble-resultatet
    results['Modell'].append('Ensemble (Voting)')
    results['Noggrannhet'].append(test_accuracy)
    
    print(f"Ensemble-modell (VotingClassifier) på {dataset_name}:")
    print(f"Test Accuracy: {test_accuracy:.4f}")
    
    # Ta bort den gamla filen om den finns
    filename = "ensemble_model_results_df2.csv"
    if os.path.exists(filename):
        try:
            os.remove(filename)
            print(f"Gammal fil borttagen: {filename}")
        except:
            print(f"Kunde inte ta bort gammal fil: {filename}")
    
    # Spara resultaten till CSV med det nya formatet
    results_df = pd.DataFrame(results)
    results_df.to_csv(filename, index=False)
    print(f"Resultat sparade i {filename} med nytt format (Modell, Noggrannhet)")
    
    return ensemble_model, test_accuracy

In [ ]:
from Ensemble_model_df2 import create_ensemble_model

ensemble_model, test_accuracy = create_ensemble_model(
    X2_train_minmax,
    y2_train,
    X2_val_minmax,
    y2_val,
    X2_test_minmax,
    y2_test,
    dataset_name="df2"
)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

def train_ensemble_model_df2(X2_train_minmax, X2_val_minmax, X2_test_minmax, 
                            y2_train, y2_val, y2_test, 
                            save_results=True, filename='ensemble_model_results_df2.csv'):
    """
    Tränar en ensemble-modell med VotingClassifier för df2-datasetet.
    
    Parametrar:
    -----------
    X2_train_minmax : array-like
        Träningsdata för df2, normaliserad med MinMaxScaler
    X2_val_minmax : array-like
        Valideringsdata för df2, normaliserad med MinMaxScaler
    X2_test_minmax : array-like
        Testdata för df2, normaliserad med MinMaxScaler
    y2_train : array-like
        Målvariabel för träningsdata
    y2_val : array-like
        Målvariabel för valideringsdata
    y2_test : array-like
        Målvariabel för testdata
    save_results : bool, default=True
        Om True, sparas resultaten i en CSV-fil
    filename : str, default='ensemble_model_results_df2.csv'
        Filnamn för att spara resultaten
        
    Returns:
    --------
    ensemble_model : VotingClassifier
        Den tränade ensemble-modellen
    """
    
    # Kombinera tränings- och valideringsdata
    X_train_val = np.vstack((X2_train_minmax, X2_val_minmax))
    y_train_val = np.concatenate((y2_train, y2_val))
    
    # Skapa individuella modeller med bästa parametrar
    rf_model = RandomForestClassifier(
        max_depth=10, 
        min_samples_leaf=4, 
        min_samples_split=10, 
        n_estimators=30,
        random_state=42
    )
    
    lr_model = LogisticRegression(
        C=100, 
        penalty='l2', 
        solver='saga',
        random_state=42,
        max_iter=1000
    )
    
    gb_model = GradientBoostingClassifier(
        learning_rate=0.1, 
        max_depth=5, 
        n_estimators=30,
        random_state=42
    )
    
    knn_model = KNeighborsClassifier(
        n_neighbors=9, 
        p=1, 
        weights='uniform'
    )
    
    # Träna individuella modeller på kombinerad tränings- och valideringsdata
    rf_model.fit(X_train_val, y_train_val)
    lr_model.fit(X_train_val, y_train_val)
    gb_model.fit(X_train_val, y_train_val)
    knn_model.fit(X_train_val, y_train_val)
    
    # Skapa ensemble-modell med VotingClassifier
    ensemble_model = VotingClassifier(
        estimators=[
            ('rf', rf_model),
            ('lr', lr_model),
            ('gb', gb_model),
            ('knn', knn_model)
        ],
        voting='soft'  # Använd sannolikheter för röstning
    )
    
    # Träna ensemble-modellen på kombinerad tränings- och valideringsdata
    ensemble_model.fit(X_train_val, y_train_val)
    
    # Utvärdera modellerna på testdata
    rf_pred = rf_model.predict(X2_test_minmax)
    lr_pred = lr_model.predict(X2_test_minmax)
    gb_pred = gb_model.predict(X2_test_minmax)
    knn_pred = knn_model.predict(X2_test_minmax)
    ensemble_pred = ensemble_model.predict(X2_test_minmax)
    
    # Beräkna noggrannhet
    rf_accuracy = accuracy_score(y2_test, rf_pred)
    lr_accuracy = accuracy_score(y2_test, lr_pred)
    gb_accuracy = accuracy_score(y2_test, gb_pred)
    knn_accuracy = accuracy_score(y2_test, knn_pred)
    ensemble_accuracy = accuracy_score(y2_test, ensemble_pred)
    
    # Skapa resultatdataframe
    if save_results:
        results = {
            'Modell': ['Random Forest', 'Logistic Regression', 'Gradient Boosting', 'KNN', 'Ensemble (Voting)'],
            'Noggrannhet': [rf_accuracy, lr_accuracy, gb_accuracy, knn_accuracy, ensemble_accuracy]
        }
        
        results_df = pd.DataFrame(results)
        results_df.to_csv(filename, index=False)
        print(f"Resultat sparade i {filename}")
        
        # Skriv ut resultat
        print("\nResultat på testdata:")
        print(f"Random Forest noggrannhet: {rf_accuracy:.4f}")
        print(f"Logistic Regression noggrannhet: {lr_accuracy:.4f}")
        print(f"Gradient Boosting noggrannhet: {gb_accuracy:.4f}")
        print(f"KNN noggrannhet: {knn_accuracy:.4f}")
        print(f"Ensemble (Voting) noggrannhet: {ensemble_accuracy:.4f}")
    
    return ensemble_model

# Exempel på användning:
# från Ensemble_model_df2 import train_ensemble_model_df2
# ensemble_model = train_ensemble_model_df2(X2_train_minmax, X2_val_minmax, X2_test_minmax, y2_train, y2_val, y2_test)

In [ ]:
# Importera ensemble-modellen
from Ensemble_model_df2 import train_ensemble_model_df2

# Kombinera tränings- och valideringsdata för att träna den slutliga modellen
ensemble_model = train_ensemble_model_df2(
    X2_train_minmax, X2_val_minmax, X2_test_minmax, 
    y2_train, y2_val, y2_test
)
